In [1]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp
import time
import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))

from utilities2015 import *
from annotation_utilities import *
from metadata import *
from data_manager import *

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

Setting environment for Local Macbook Pro


In [2]:
stack = 'MD589'

In [4]:
# visualize thumbnail volume itself

# tb_volume = load_thumbnail_volume(stack)
tb_volume = DataManager.load_intensity_volume(stack=stack)

launch_vtk([actor_volume(tb_volume[:,:,tb_volume.shape[2]/2:], 'tb')], 
           window_name='Thumbnail Volume %(stack)s' % {'stack': stack})

In [24]:
# visualize thumbnail volume with annotation

In [4]:
contour_df, _ = DataManager.load_annotation_v3(stack=stack, annotation_rootdir=annotation_midbrainIncluded_v2_rootdir)

'No object named structures in the file'


Annotation has no structures.


In [5]:
downsample_factor = 32
xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
voxel_z_size = section_thickness / xy_pixel_distance_downsampled

In [6]:
anchor_filename = metadata_cache['anchor_fn'][stack]
sections_to_filenames = metadata_cache['sections_to_filenames'][stack]
filenames_to_sections = {f: s for s, f in sections_to_filenames.iteritems() 
                        if f not in ['Placeholder', 'Nonexisting', 'Rescan']}

In [7]:
import cPickle as pickle
Ts = pickle.load(open(thumbnail_data_dir + '/%(stack)s/%(stack)s_elastix_output/%(stack)s_transformsTo_anchor.pkl' % dict(stack=stack), 'r'))

Ts_inv_downsampled = {}
for fn, T0 in Ts.iteritems():
    T = T0.copy()
    T[:2, 2] = T[:2, 2] * 32 / downsample_factor
    Tinv = np.linalg.inv(T)
    Ts_inv_downsampled[fn] = Tinv

In [8]:
def collect(matched_contours, convert_to='aligned'):
    """
    Convert matched annotation dataframe entries to contours as a list of 3d array.
    """
    
    contours_3d = []
    
    for cnt_id, cnt in matched_contours.iterrows():

        fn = cnt['filename']
        
        if fn not in filenames_to_sections:
            continue
            
        section = filenames_to_sections[fn]
        z = voxel_z_size * (section-1)

        if convert_to == 'aligned':
        
            n = len(cnt['vertices'])
            vertices_on_aligned = np.dot(Ts_inv_downsampled[fn], np.c_[cnt['vertices']/downsample_factor, np.ones((n,))].T).T[:, :2]

            vertices_on_aligned_3d = np.c_[vertices_on_aligned, z*np.ones((n,))]
            contours_3d.append(vertices_on_aligned_3d)
        
    return contours_3d

In [9]:
from skimage.draw import polygon_perimeter

In [10]:
crop_xmin, crop_xmax, crop_ymin, crop_ymax = metadata_cache['cropbox'][stack]
print 'crop:', crop_xmin, crop_xmax, crop_ymin, crop_ymax

tb_vol_xmin, tb_vol_xmax, tb_vol_ymin, tb_vol_ymax, tb_vol_zmin, tb_vol_zmax = \
np.loadtxt(os.path.join(VOLUME_ROOTDIR, stack, '%(stack)s_down%(downsample)dVolume_bbox.txt') % \
           dict(stack=stack, downsample=32), dtype=np.int)
print 'tb_vol:', tb_vol_xmin, tb_vol_xmax, tb_vol_ymin, tb_vol_ymax, tb_vol_zmin, tb_vol_zmax

crop: 569 1053 140 512
tb_vol: 0 484 0 372 123 503


In [11]:
# structures = set(contour_df['name'])

paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures


In [12]:
tb_volume = load_thumbnail_volume(stack)
tb_vol_ydim, tb_vol_xdim, tb_vol_zdim = tb_volume.shape

In [13]:
tb_volume = tb_volume.astype(np.float32)

In [14]:
# Make contours have value x.5 (between integers)
structure_to_value = {name_u: i % len(structures) + 1.5 for i, name_u in enumerate(structures)}

In [15]:
for i, name_u in enumerate(structures):

    matched_contours = contour_df[(contour_df['name'] == name_u)]
    contours_3d = collect(matched_contours)

    for cnt in contours_3d:

        vs = cnt.astype(np.int)

        polygon_ys, polygon_xs = polygon_perimeter(vs[:,1], vs[:,0])

        ys = polygon_ys - crop_ymin - tb_vol_ymin
        xs = polygon_xs - crop_xmin - tb_vol_xmin

        valid = (ys >= 0) & (ys < tb_vol_ydim) & (xs >= 0) & (xs < tb_vol_xdim)

        z = int(np.floor(vs[0,2])) - tb_vol_zmin

        tb_volume[ys[valid], xs[valid], z] = structure_to_value[name_u]

In [42]:
# All contours in red
# tb_colors = {i % len(structures) + 1.5: (1.,0,0) for i, name_u in enumerate(structures)}

# Different structures in different color
# tb_colors = {i % len(structures) + 1.5: np.random.randint(0, 255, (3,))/255. for i, name_u in enumerate(structures)}
tb_colors = {i % len(structures) + 1.5: np.array(high_contrast_colors[np.random.randint(0, len(high_contrast_colors), 1)])/255. for i, name_u in enumerate(structures)}

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


In [17]:
launch_vtk([actor_volume(tb_volume, 'tb', tb_colors=tb_colors)])

In [18]:
cut_z = 100
cut_xmin = 50
cut_xmax = 500

In [19]:
tb_volume_half = tb_volume.copy()
tb_volume_half[:, :, :cut_z] = 0
tb_volume_half[:, :cut_xmin, :] = 0
tb_volume_half[:, cut_xmax:, :] = 0

launch_vtk([actor_volume(tb_volume_half, 'tb', tb_colors=tb_colors)])

In [31]:
tb_volume_sectioned = np.zeros_like(tb_volume)
interval = 10
# for z in range(tb_volume_sectioned.shape[2]/2, tb_volume_sectioned.shape[2], interval):
for z in range(cut_z, tb_volume_sectioned.shape[2], interval):
    tb_volume_sectioned[:, :, z:z+3] = tb_volume[:,:,z:z+3]

In [21]:
launch_vtk([actor_volume(tb_volume_sectioned, 'tb', tb_colors=tb_colors)])

In [43]:
tb_volume_piece = tb_volume[:,cut_xmin:cut_xmax+1,:cut_z+5].copy()
tb_volume_piece[:, :, :cut_z] = 0

launch_vtk([actor_volume(tb_volume_piece, 'tb', tb_colors=tb_colors, origin=(50,0,0), tb_opacity=.4),
           actor_volume(tb_volume_sectioned, 'tb', tb_colors=tb_colors, tb_opacity=.01)])

In [260]:
# Plot VTK transfer function

# compositeOpacity = vtk.vtkPiecewiseFunction()
# compositeOpacity.AddPoint(0.0, 0.)
# for v, c in sorted(tb_colors.items()):
#     vl = v - .5
#     vr = v + .5
#     cp1 = vl-.25
#     cp2 = vr-.25
#     compositeOpacity.AddPoint(cp1, .5*cp1/200., .5, 1.)
#     compositeOpacity.AddPoint(v, 1., .5, 1.)
#     compositeOpacity.AddPoint(cp2, .5*cp2/200., .5, 1.)
# compositeOpacity.AddPoint(vr, .5*vr/200.)
# compositeOpacity.AddPoint(240., 0.05)
# compositeOpacity.AddPoint(255.0, 0.05)

# q = np.array([compositeOpacity.GetValue(v) for v in np.arange(0, 255, .1)])

# plt.figure(figsize=(20,5));
# plt.plot(np.arange(0, 255, .1), q);